# Business Understanding

# Data Understanding

# Data Preparation

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt


In [16]:
import sqlite3
conn = sqlite3.connect('im.db')


OperationalError: unable to open database file

In [15]:
Movie_Gross = pd.read_csv("bom.movie_gross.csv.gz")

# Exploratory Data Analysis

In [12]:
pd.read_sql("""
SELECT * FROM movie_basics;
"""
, conn)

DatabaseError: Execution failed on sql '
SELECT * FROM movie_basics;
': no such table: movie_basics

# Conclusions

## Limitations

## Recommendations

## Next Steps